In [ ]:
// google-map-test
show('<iframe src="https://www.google.com/maps/embed?pb=!1m10!1m8!1m3!1d2793.816117697252!2d39.74576502281489!3d54.628038298661046!3m2!1i1024!2i768!4f13.1!5e1!3m2!1sru!2sru!4v1730885538275!5m2!1sru!2sru" width="100%" height="600" style="margin: 0 auto; border:0;" allowfullscreen="" loading="lazy" referrerpolicy="no-referrer-when-downgrade"></iframe>')

In [ ]:
// video-test
show('<iframe width="720" height="405" src="https://rutube.ru/play/embed/ad722c6958696f78f1908384f85120e8/" style="box-shadow: 0px 5px 10px 0px rgba(0, 0, 0, 0.5); padding: 16px;margin: 16px auto;" frameBorder="0" allow="clipboard-write; autoplay" webkitAllowFullScreen mozallowfullscreen allowFullScreen></iframe>')


In [ ]:
import {torus} from '/web/oda/apps/neuronet/genius/torus/torus.js';
import {nn} from '/web/oda/apps/neuronet/genius/neuro-module/neuro-module.js';
import {tensor} from '/web/oda/apps/neuronet/genius/torus/torus.js';
import '/web/oda/apps/neuronet/genius/neuro-module/loss-chart/loss-chart.js';

In [ ]:
torus.prototype.updateParams = function () {
        if (!this.isParam) return;


        let lr = torus.LEARNING_RATE
        for(let i = 0; i<this.data.length; i++){
            let change = this.grad[i] * lr //* torus.generator();
            this.data[i] += change;
        }


            // for(let i = 0; i<this.data.length; i++){
            //     let gamma = torus.generator();
            //     let prev = this.prev[i] * gamma;
            //     lr = 1 - gamma;
            //     let change = prev + this.grad[i] * lr;
            //     this.data[i] += change;
            //     this.prev[i] = change
            // }


            // let lr = torus.LEARNING_RATE || .01
            // for(let i = 0; i<this.data.length; i++){
            //     let prev = this.prev[i];
            //     let change = this.grad[i] * lr + prev * .5;
            //     this.data[i] += change;
            //     this.prev[i] = change;
            // }


            //

            // for(let i = 0; i<this.data.length; i++){
            //     const lr = torus.generator() / 3;
            //     const lambda = 1 - lr;
            //     let prev = this.prev[i];
            //     let change = (this.grad[i] + lambda * prev) * lr;
            //     this.data[i] += change;
            //     this.prev[i] = change * lr;
            //
            // }

        this.clearGrad();
    }

In [ ]:
MNIST =  class MNIST extends nn.NeuroModule{
    constructor(dim = 28) {
        super(arguments);
    }    
    __init__(){
        let netSape = [this.dim * this.dim, (this.dim * this.dim) / 4, 10]
        this.layer1 = nn.Linear(netSape[0], netSape[1], true)
        this.layer2 = nn.Linear(netSape[1], netSape[2], true)
    }    
    forward(x, target, back = true){
        x = tensor.from(x)
        x = this.layer1(x)
        x = x.sigm()
        x = this.layer2(x)
        x = x.softmax();
        if(target){
            const loss = x.crossEntropy(target);
            let loss_ = loss.get()
            // if (!Number.isNaN(loss_)) {
                if(!back)
                    return loss

                this.losses.push(loss_);
                loss.back();
            // }
            // else console.log ('loss isNaN')
        }
        return x
    }
}

In [ ]:
datasetRaw = await ODA.loadJSON('/web/oda/apps/neuronet/mnist/mnist.json');
targets = targets = Array(10).fill(0).map((_,i)=> {
    let t =  Array(10).fill(0)
    t[i]=1
    return t
})

In [ ]:
getTrain = (dataset = datasetRaw, batch=5, test_size=0.2) => {
        let [result, res] = [{inputs:[],targets:[]},[]]

        let suBatch = Array(10).fill(0).map((_,i)=>i) //.sort(()=>Math.random() - 0.5)
        suBatch.forEach(n=>{
            // let target  = Array(10).fill(0);
            // target[n] = 1
            let target  = targets[n]
            for (let j=0; j<batch; j++) {
                let br = Math.floor(dataset[n].length * (1-test_size))
                let k = Math.floor(Math.random() * br)
                res.push([dataset[n][k].flat(),target])
            }
        })

        res.sort(()=>Math.random() - 0.5)
        res.forEach ( ([inp,tar]) => {result.inputs.push(inp); result.targets.push(tar)} )
        return result
    }

In [ ]:
ODA({is: 'oda-mnist-input', imports: '@oda/button', template: /*html*/ `
        <style> :host { @apply --vertical; } </style>
        <svg style='margin:1em;' width='281' xmlns='http://www.w3.org/2000/svg' viewBox='0 0 281 281' preserveAspectRatio="xMidYMin meet">
            <rect x="0" y="0" width="281" height="281" fill='#4d85cf'/>
            <rect ~for='rects' :x="$for.item?.x" :y="$for.item?.y" width="9" height="9" :fill='$for.item?.v?"#070637":"#a9caff"'
                    @mousemove="_mouseMove($event, $for.item)"/>
        </svg>
        <div horizontal style='margin:-1em 1em 1em 1em; width:281px; background: #a9caff;'>
            <oda-button :disabled icon-size="18" :icon="drawBoll?'bootstrap:vector-pen':'bootstrap:eraser-fill'" @tap='drawBoll=!drawBoll'></oda-button>
            <i flex></i>
            <oda-button :disabled icon-size="18" icon="bootstrap:trash3-fill" @tap='input= new Array(28*28).fill(0)'></oda-button>
        </div> `,
    get rects() { return this.input.map((v,i)=> Object( {v, x:(i%28)*10+1, y:(~~(i/28))*10+1}) ) },
    input: { $def: new Array(28*28).fill(0) },
    drawBoll:true, disabled:false,
    _mouseMove(x,e) {
        if (!x.buttons) return
        if (e.v===this.drawBoll) return
        this.input[Math.floor(e.y/10)*28 + Math.floor(e.x/10)] = this.drawBoll
    },
});

ODA({is: 'oda-mnist-output', imports: '@oda/button', template: /*html*/ `
    <svg style='margin:1em;' width="281" height="281" xmlns='http://www.w3.org/2000/svg' viewBox='0 0 281 281'>
        <rect x="0" y="0" width="281" height="281" fill='#a9caff'/>
        <g ~for='result' :transform="'translate(0,'+ ($for.index * 28) +')'">
            <rect x="2" y="2" :width="276*$for.item" height="24" :fill='maxIndex===$for.index? "#ff00ff55" : "#4d85cf55"'/>
            <text x="265" y="16" font-size='16' dominant-baseline="middle" fill="#070637">{{$for.index}}</text>
            <text x="5" y="16" font-size='16' dominant-baseline="middle" fill="#070637">{{($for.item*100).toFixed(2)}} %</text>
        </g>
        <text x="150" y="160" font-size='200' dominant-baseline="middle" text-anchor="middle" fill="#070637">{{maxIndex}}</text>
    </svg> `,
    result:[.1, .01, .1, .01, .28, .1, .1, .1, .1, .1],
    get maxIndex() { return this.result.reduce((a,b,i,c)=>c[a]>b?a:i,0) },
});

ODA({is: 'oda-mnist-visual', template: /*html*/ `
    <div horizontal>
        <oda-mnist-input ::input :disabled ></oda-mnist-input>
        <oda-mnist-output :result='output' ></oda-mnist-output>
        <oda-loss-chart style='margin:1em;' flex :data='loss'></oda-loss-chart>
    </div>`,
    input: { $def: new Array(28*28).fill(0) }, disabled:false,
    output: [.05, .01, .15, .01, .28, .1, .1, .1, .1, .1],
    loss:[]
});  

In [ ]:
visual = this.createElement('oda-mnist-visual', {})
// show(visual)


In [ ]:
mnist = new MNIST()


trane = () => {
        let rez, dat = getTrain(undefined, 1);
        for (let i=0; i<dat.inputs.length; i++) {
            rez = mnist(dat.inputs[i], dat.targets[i])
            if (i==(dat.inputs.length-1)) {
                visual.input = dat.inputs[i].map(a=>a)
                visual.output = Array.from(rez.data)
                
        //         // this.lastLoss = mnist.loss.toFixed(4)
                visual.loss = mnist.losses
            }
        }
}
// trane()


In [ ]:
ODA({is: 'oda-mnist-play', imports: '@oda/button', template: /*html*/ `
    <oda-button :label='go?"pause":"play"' :icon='go?"av:pause":"av:play-arrow"' @tap="_go()"></oda-button>`,
    go:false,
    _go(){ this.go=!this.go; this.steps(); },
    steps(){ if (this.go) this.async(() => {trane(); this.steps();}, 100) },
})
btnPlay = this.createElement('oda-mnist-play', {})


In [ ]:
// show(visual)
show("<iframe src='https://www.google.com/maps/embed?pb=!1m14!1m12!1m3!1d476.2836683410046!2d39.74605682577753!3d54.627878585674765!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!5e0!3m2!1sru!2sru!4v1730879578803!5m2!1sru!2sru' width='800' height='600' style='margin: 0 auto; border:0;' allowfullscreen='' loading='lazy' referrerpolicy='no-referrer-when-downgrade'></iframe>")
show("<h1 style='color: red;'>ЭТО ЗАГОЛОВОК H1</h1>", "<button>ЖМИ МЕНЯ</button>")
show('<h2>ЭТО ПОДЗАГОЛОВОК H2<h2>',btnPlay,'<h3>ЭТО ПОДЗАГОЛОВОК H3<h3>')
show(visual)

<h2>!!<h2>

In [ ]:
ODA({is: 'oda-mnist', imports: '@oda/button', template: /*html*/ `
    <style>
        :host {max-width:686px; margin:auto; display:grid; grid-template-columns:1fr 1fr; grid-template-areas: 'input output' 'loss loss'; }
        .block {border: 1px solid #4d85cf; padding:10px; margin:10px;}
        .box { margin:10px}
        .block h3 {margin:0 10px; display:flex; color:#070637; align-items: center;}
        .block h3 i {font-size:0.8em; opacity:0.8;}
        oda-button {border: 1px solid #4d85cf; margin:10px;}
        h3  oda-button {margin:1px;padding:1px;}
        .block h3 input {border:0; width:1em; outline:none; font-style:italic; font-weight:bold; font-size:15px}
    </style>
    <div class='block' style='grid-area:input;'>
        <h3>input:<i flex></i>
            <oda-button :disabled='go' icon-size="18" icon="bootstrap:trash3-fill" @tap='input=new Array(28*28).fill(0)'></oda-button>
            <oda-button :disabled='go' icon-size="18" :icon="drawBoll?'bootstrap:vector-pen':'bootstrap:eraser-fill'" @tap='drawBoll=!drawBoll'></oda-button>
        </h3>
        <svg class='box' width="281" height="281" xmlns='http://www.w3.org/2000/svg' viewBox='0 0 281 281'>
            <rect x="0" y="0" width="281" height="281" fill='#4d85cf'/>
            <rect ~for='rects' :x="$for.item?.x" :y="$for.item?.y" width="9" height="9" :fill='$for.item?.v?"#070637":"#a9caff"'
                    @mousemove="_mouseMove($event, $for.item)"/>
        </svg>
    </div>
    <div class='block' style='grid-area:output;'>
        <h3><b>output:</b><i flex></i><i>target:<input @input='_inTarget' :value='target'/></i>
            <oda-button :disabled='go' icon-size="18" icon="carbon:ibm-watson-openscale" @tap='_dopTrain()'></oda-button>
        </h3>
        <svg class='box' width="281" height="281" xmlns='http://www.w3.org/2000/svg' viewBox='0 0 281 281'>
            <rect x="0" y="0" width="281" height="281" fill='#a9caff'/>
            <g ~for='result' :transform="'translate(0,'+ ($for.index * 28) +')'">
                <rect x="2" y="2" :width="276*$for.item" height="24" :fill='maxIndex===$for.index? "#ff00ff55" : "#4d85cf55"'/>
                <text x="265" y="16" font-size='16' dominant-baseline="middle" fill="#070637">{{$for.index}}</text>
                <text x="5" y="16" font-size='16' dominant-baseline="middle" fill="#070637">{{($for.item*100).toFixed(2)}} %</text>
            </g>
            <text x="150" y="160" font-size='200' dominant-baseline="middle" text-anchor="middle" fill="#070637">{{maxIndex}}</text>
        </svg>
    </div>
    <div class='block' style='grid-area:loss;'>
        <h3><b>loss: {{lastLoss}} </b><i flex></i><i>generation: {{generation}}</i></h3>
        <svg class='box' :width :height xmlns='http://www.w3.org/2000/svg' :viewBox="'0 0 ' + width + ' ' + height">
            <rect x="0" y="0" :width :height fill='#a9caff'/>
            <path :d='pathD' fill="none" stroke="#070637" stroke-width="2"/>
        </svg>
    </div>
    <oda-button :label='go?"pause":"play"' :icon='go?"av:pause":"av:play-arrow"' @tap="_go()"></oda-button>
    `,
    _inTarget(e) {
        if (!e.data) return
        let old = this.target
        if ((47<e.data.charCodeAt())&(e.data.charCodeAt()<58)) e.target.value =  this.target = e.data
        else  e.target.value =  this.target = old
    },
    _dopTrain(){
        this.mnist.generation+=1
        let rez = this.mnist(this.input, targets[this.target])
        this.result = rez.data
    },
    iconFromChar(c) { /// 🗑️✍🏻✒️♻💊✍🏿
        return /*html*/ `data:image/svg+xml,
            <svg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 100 100'>
                <text y='.9em' x='-8' font-size='90'>${c}</text>
            </svg>`},
    get rects() {
        return this.input.map((v,i)=> {
            let [x,y] = [i%28, ~~(i/28)].map(a=>a*10+1)
            return {x,y,v}
        } )
    },
    get maxIndex() { return this.result.reduce((a,b,i,c)=>c[a]>b?a:i,0) },

    get pathD(){
        if (this.pointsL.length<2) return ''
        let padding = Math.min(this.width,this.height)/10
        let [w,h] = [this.width,this.height].map(x=>x-padding*2)
        let dx = w/(this.pointsL.length-1)
        let [min,max] = [Math.min(...this.pointsL),Math.max(...this.pointsL)]
        let points = this.pointsL.map((v,i)=>[i*dx+padding,h-(v-min)/(max-min)*h+padding])
        let rez = 'M ' + points[0].join(' ') + ' '
        if (this.bezier) rez+=  'Q ' + points[0].join(' ') + ', ' + points[1].join(' ') + ' '
        else  rez += 'L ' + points[1].join(' ') + ' '
        points.slice(2).forEach(e => rez += (this.bezier?'T ':'L ') + e.join(' ') + ' ' )
        return rez
    },

    get pointsL() {
        let [lL,lP] = [this.loss.length,this.maxPoint]
        if (lL<lP) return this.loss
        let newP = Array(lP-1).fill(0).map((_,i)=>{
            let idx = Math.floor(i*(lL/(lP-1)))
            let dL = Math.floor(lL/(lP-1))+1
            return this.loss.slice(idx,idx+dL).reduce((a,c)=>a+c)/dL
        })
        newP.push(this.loss[lL-1])
        return newP
    },
    bezier:false,

    go:false,
    _go(){
        this.go = !this.go
        this.steps()
    },

    width:281*2+10*6+2,
    height:281,
    maxPoint:100,

    input: {
        $def: new Array(28*28).fill(0)
    },
    result:[],
    loss:{
        $def:[]//new Array(20000).fill(0).map(_=> Math.random()*5-2)
    },
    drawBoll:true,
    _mouseMove(x,e) {
        if (!x.buttons) return
        if (e.v===this.drawBoll) return

        let idx = Math.floor(e.y/10)*28 + Math.floor(e.x/10)
        this.input[idx] = this.drawBoll
        this.result = Array.from(this.mnist(this.input).data)

        // console.log(e.y,e.x ,e.y*28+e.x ,this.input[e.y*28+e.x])
    },

    mnist: new MNIST(),
    getTrain:getTrain,

    step() {
        let rez, mnist = this.mnist, dat = this.getTrain(undefined, 1);
        for (let i=0; i<dat.inputs.length; i++) {
            rez = mnist(dat.inputs[i], dat.targets[i])
            if (i==(dat.inputs.length-1)) {
                this.input = dat.inputs[i].map(a=>a)
                this.result = Array.from(rez.data)
                // this.lastLoss = mnist.loss.toFixed(4)
                this.generation = mnist.losses.length
            }
        }
        this.loss= mnist.losses

    },
    steps(){
        if (!this.go) return;
        this.async(() => {
            this.step()
            this.steps()
        }, 100)
    },
    lastLoss:'',
    generation:0,
    target:'',
})




In [ ]:
// тест binarize
function binarize (grad_i) {
    let p = Math.max(0,Math.min(1,(grad_i+1)/2)) // σ(x) = hard sigmoid
    return p>Math.random() // +1 with probability p = σ(x), -1 otherwise.
}

let grad = -0.7
print (`σ(x) = ${Math.max(0,Math.min(1,(grad+1)/2))} = ${Math.max(0,Math.min(1,(grad+1)/2))*100} % ` )

let n = 1000
let rezz = Array(n).fill(grad).map(g => binarize(g)).reduce((akk,b) => { b? akk.true++:akk.false++; return akk },{true:0,false:0} )

print(`true: ${rezz.true}, (${rezz.true/n*100} %) `)

σ(x) = 0.15000000000000002 = 15.000000000000002 % 

true: 135, (13.5 %) 

In [ ]:
// тест сложения
v = Array(15).fill(0).map(a =>  (Math.random()>0.5)?0:1  ) // [0,1,1,1,0]
mul = v.filter(b=>b).reduce((akk,b) => akk*(-1), 1 )>0
print (mul)
result = 'return (' + v.map((_,i)=>{
                    return '(v['+i+'] - 1)';
            }).filter(l=>l).join(` + `) + ')&1;\n';
print(result)
const fn = new Function('v',result);
print(fn(v))
// print(`in= ${v.join('')}  mul =`fn(v))

false

return ((v[0] - 1) + (v[1] - 1) + (v[2] - 1) + (v[3] - 1) + (v[4] - 1) + (v[5] - 1) + (v[6] - 1) + (v[7] - 1) + (v[8] - 1) + (v[9] - 1) + (v[10] - 1) + (v[11] - 1) + (v[12] - 1) + (v[13] - 1) + (v[14] - 1))&1;


0